In [1]:
import pandas as pd
import numpy as np
import uuid
import pickle
import os
import configparser
import pymysql
import multiprocessing as mp
import sys
from imp import reload
# from pyspark.ml.fpm import FPGrowth
from Code import data_generator_new
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [ ]:
reload(data_generator_new)

### 数据探查

In [ ]:
data_raw = pd.read_sql("select comp_id, label_name, classify_id, label_type, label_type_num from company_tag_info_latest0703", con=data_generator_new.db)

In [ ]:
len(data_raw)

In [ ]:
comp_id_dict = data_raw.comp_id.drop_duplicates().reset_index(drop=True).reset_index() \
    .rename(index=str, columns={"index": "comp_int_id"})
data_raw = data_raw.merge(comp_id_dict, how="left", left_on="comp_id", right_on="comp_id")

In [ ]:
data_raw.head(3)

In [ ]:
ctag_table = data_raw[data_raw.classify_id != 4]

In [ ]:
ctag_agg = nctag_table.groupby("label_name").agg({"comp_id": "count"})

In [ ]:
ctag_agg.sort_values(by="comp_id", ascending=False)

In [ ]:
nctag_table = data_raw[data_raw.classify_id == 4]

In [ ]:
nctag_agg = nctag_table.groupby("label_name").agg({"comp_id": "count"}).reset_index()

In [ ]:
nctag_agg["word_len"] = nctag_agg.label_name.apply(lambda x: len(str(x)))

In [ ]:
nctag_agg.sort_values(by="comp_id", ascending=False).head(100)

In [ ]:
nctag_agg[nctag_agg.comp_id >= 150].groupby("word_len").agg({"label_name": "count"})

In [ ]:
nctag_agg[(nctag_agg.comp_id >= 150) & (nctag_agg.word_len > 3)]

In [ ]:
nctag_to_keep = nctag_agg[(nctag_agg.comp_id >= 150) & (nctag_agg.comp_id <= 1000000) & (nctag_agg.word_len >= 4)]

In [ ]:
nctag_to_keep

In [ ]:
nctag_table = nctag_table[nctag_table.label_name.isin(nctag_to_keep.label_name)]

In [ ]:
nctag_table.head(2)

In [ ]:
ctag_table.head(2)

In [ ]:
pwd

### 检测comp_tag函数

In [ ]:
data_generator_new.comp_tag()

### 检测data_aggregator函数，返回概念、非概念各自聚合公司的结果，以及公司标签关联、公司总数

In [4]:
data_generator_new.data_aggregator()

tag_comps_aggregated_for_spark saved!
tag_points appended!
company-tag link count: 28913615
comp_tag_relations saved


0

### 计算公司标签全集

In [ ]:
data_generator_new.properties()

### 计算标签关联

### 全流程测试

In [ ]:
!python Code/pipline_new.py

### 智能网联汽车关联的top1000概念非概念标签

In [ ]:
relations = pd.read_csv("../Data/Output/Tag_graph/all_relations.csv", header=None)

In [ ]:
relations.columns = ["src_id", "target_id", "rel_value", "rel_type"]

In [ ]:
relations.head(1)

In [ ]:
tag_dict = pickle.load(open("../Data/Output/Tag_graph/tag_dict.pkl", "rb"))
tag_dict_reverse = dict(zip(tag_dict.values(), tag_dict.keys()))

In [ ]:
target_tag = tag_dict.get("智能网联汽车")

In [ ]:
target_tag

In [ ]:
linked_ctag_top_1000 = relations[(relations.src_id == target_tag) & (relations.rel_type == "ctag_ctag")].sort_values(by="rel_value", ascending=False)

In [ ]:
len(linked_ctag_top_1000)

In [ ]:
_linked_ctag_top_1000 = relations[(relations.src_id == target_tag) & (relations.rel_type == "ctag_ctag")].sort_values(by="rel_value", ascending=False)

In [ ]:
len(_linked_ctag_top_1000)

In [ ]:
linked_nctag_top_1000 = relations[(relations.target_id == target_tag) & (relations.rel_type == "ctag_ctag")].sort_values(by="rel_value", ascending=False)[:1000]

In [ ]:
len(linked_nctag_top_1000)

In [ ]:
_linked_nctag_top_1000 = relations[(relations.src_id == target_tag) & (relations.rel_type == "ctag_ntag")].sort_values(by="rel_value", ascending=False)[:1000]

In [ ]:
linked_ctag_top_1000["target_tag_name"] = linked_ctag_top_1000.targe_id.apply(lambda x: tag_dict_reverse.get(x))[:1000]

In [ ]:
linked_nctag_top_1000["target_tag_name"] = linked_nctag_top_1000.targe_id.apply(lambda x: tag_dict_reverse.get(x))[:1000]

In [ ]:
df = spark.createDataFrame([
    (0, "a"),
    (1, "a"),
    (1, "b"),
    (1, "c"),
    (2, "a"),
    (2, "b"),
    (2, "b"),
    (2, "b"),
    (2, "c"),
    (0, "a"),
    (1, "b"),
    (1, "b"),
    (2, "cc"),
    (3, "a"),
    (4, "a"),
    (5, "c")
], ["id", "category"])

In [ ]:
df.show()

In [ ]:
pivoted = df.groupBy("id").pivot("category").count().na.fill(0)

In [ ]:
pivoted.show()

In [ ]:
from pyspark.ml.feature import MinHashLSH

In [ ]:
%run minhash.py

# MinHash test

In [2]:
from pyspark.ml.feature import CountVectorizer, MinHashLSH
from pyspark.sql.functions import col

In [3]:
%%time
tag_comps_aggregated = pickle.load(open("../Data/Output/Tag_graph/tag_comps_aggregated.pkl", "rb"))

CPU times: user 4.73 s, sys: 1.33 s, total: 6.06 s
Wall time: 6.03 s


In [4]:
tag_comps_aggregated.head(3)

,comp_int_id,type,tag_uuid
0,"{1671170, 5406730, 1179663, 3735567, 262169, 7...",1,a2f6e88e9a65510ba21ce44965ca7b36
1,"{1671170, 3637262, 589847, 2129945, 851998, 16...",1,b5426c809798598cb5735dcb6a7c17c8
2,"{2275332, 5376008, 1241096, 3989514, 5883914, ...",1,299f1692f39a54989bb7a754533d7e3e


In [5]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [6]:
tag_comps_aggregated["comp_int_id_list"] = tag_comps_aggregated.comp_int_id.apply(lambda x: [str(w) for w in x])

In [7]:
tag_comps_aggregated_spark = spark.createDataFrame(tag_comps_aggregated[["tag_uuid", "comp_int_id_list"]])

In [8]:
tag_comps_aggregated_spark.show(3)

+--------------------+--------------------+
|            tag_uuid|    comp_int_id_list|
+--------------------+--------------------+
|a2f6e88e9a65510ba...|[1671170, 5406730...|
|b5426c809798598cb...|[1671170, 3637262...|
|299f1692f39a54989...|[2275332, 5376008...|
+--------------------+--------------------+
only showing top 3 rows



In [12]:
tag_comps_aggregated_spark.persist()

DataFrame[tag_uuid: string, comp_int_id_list: array<string>]

In [9]:
%%time
print("Sparse Vectorizing")
cv = CountVectorizer(inputCol="comp_int_id_list", outputCol="sparse_vector", minDF=1, minTF=1, vocabSize=7000000)
cv_model = cv.fit(tag_comps_aggregated_spark)

Sparse Vectorizing
CPU times: user 25.1 ms, sys: 4.43 ms, total: 29.6 ms
Wall time: 52.9 s


In [10]:
%%time
tag_comps_sparse = cv_model.transform(tag_comps_aggregated_spark)
tag_comps_sparse.show(3)
mh = MinHashLSH(inputCol="sparse_vector", outputCol="hashes", seed=12345, numHashTables=256)
print("Fitting lsh model")
mh_model = mh.fit(tag_comps_sparse)
tag_comps_sparse.persist()

+--------------------+--------------------+--------------------+
|            tag_uuid|    comp_int_id_list|       sparse_vector|
+--------------------+--------------------+--------------------+
|a2f6e88e9a65510ba...|[1671170, 5406730...|(6663224,[104,189...|
|b5426c809798598cb...|[1671170, 3637262...|(6663224,[26,104,...|
|299f1692f39a54989...|[2275332, 5376008...|(6663224,[333,711...|
+--------------------+--------------------+--------------------+
only showing top 3 rows

Fitting lsh model
CPU times: user 53.4 ms, sys: 11.2 ms, total: 64.6 ms
Wall time: 2min 1s


In [ ]:
%%time
print("Similarity cross join")
result = mh_model.approxSimilarityJoin(tag_comps_sparse, tag_comps_sparse, 0.9, distCol="JaccardDistance")
result.show(3)
result_clean = result.select(
    col("datasetA.id").alias("idA"),
    col("datasetB.id").alias("idB"),
    col("JaccardDistance"))
result.clean.show(3)

Similarity cross join
